In [ ]:
!pip install -q sentence-transformers langchain chromadb pypdf faiss-cpu \
langchain_community scikit-learn numpy mistralai langchain-mistralai


In [ ]:
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import PromptTemplate
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

import os
from getpass import getpass


In [ ]:
os.environ["MISTRAL_API_KEY"] = getpass("Enter Mistral API Key: ")


In [ ]:
!pip install pypdf


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/FUNDAMENTALS OF MANAGEMENT.pdf")   # upload your PDF to colab
docs = loader.load()
len(docs)


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(docs)
len(chunks)


In [ ]:
embeddings = MistralAIEmbeddings(model="mistral-embed")

vector_db = FAISS.from_documents(chunks, embedding=embeddings)
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


In [ ]:
template = """
You are a Memory-Augmented RAG assistant.

Conversation History:
{chat_history}

Relevant Retrieved Context:
{context}

User Query:
{question}

Provide a context-aware answer. ALWAYS use history if relevant.
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "context", "question"],
    template=template
)


In [ ]:
llm = ChatMistralAI(model="mistral-large-latest", temperature=0)

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt}
)


In [ ]:
while True:
    q = input("You: ")
    if q.lower() in ["exit", "quit"]:
        break
    resp = rag_chain.invoke({"question": q})
    print("Bot:", resp["answer"])
